In [1]:
# importing relevant packages 

import torch 
from torchvision import models
import pandas as pd
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn 
import glob
import os
import cv2
import matplotlib.pyplot as plt 
import torchvision.transforms as transforms
from torchsummary import summary
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm
import json
from PIL import Image
import random
import albumentations as A
import glob
from shapely.geometry import Polygon

/home/dell/Desktop/xxxx/Uni/Kirigami_project/Keypoint_detection_notebooks/.Keypoint_detection_notebooks/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
class VGG_model(nn.Module):
    def __init__(self):
        super(VGG_model, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
            nn.Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
            nn.Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
            nn.Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
            nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        )
        
        self.avgpool = nn.Sequential(
            nn.Conv2d(512,512, kernel_size=3, padding='same'),
            nn.LeakyReLU(0.1,inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(512,50, kernel_size=3, padding='same'),
            nn.LeakyReLU(0.1,inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.AdaptiveAvgPool2d(output_size=(8,8))
        )
        
        self.classifier = nn.Sequential(
            nn.Linear(3200, 300),
            nn.LeakyReLU(0.1,inplace=True),
            nn.Dropout(0.3),
            nn.Linear(300, 32),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x
        

In [3]:
model = VGG_model()
model.load_state_dict(torch.load('./Model_VGG_4_5_6_2000_16300_epoch'))
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.eval()
model = model.to(device)

In [4]:
def predict_keypoints(img):
    img = cv2.resize(img, (224,224))
    img_tensor = transforms.ToTensor()(img)
    img_mean = img_tensor.mean(dim = (1,2))
    img_std = img_tensor.std(dim = (1,2))
        
    img_normalised = transforms.Normalize(img_mean, img_std)(img_tensor)
    img_normalised = img_normalised.to(device)
    
    key_points = model(img_normalised[None]).flatten().detach().cpu().numpy()
    
    return key_points

In [5]:
def plot_keypoints(img, keypoints):                                                             

    plt.imshow(img)

    x_points = keypoints[0::2]
    y_points = keypoints[1::2]
    plt.scatter(x_points*img.shape[1], y_points*img.shape[0], s = 4, c=(1,0,0))
    plt.show()

In [6]:
# dataset_names = ['Grasp_dataset_orange', 'Grasp_dataset_pear', 'Grasp_dataset_banana', 'Grasp_dataset_plum', 'Grasp_dataset_egg', 'Grasp_dataset_strawberry', 'Grasp_dataset_chicken', 'Grasp_dataset_bayberry', 'Grasp_dataset_redgrape', 'Grasp_dataset_pistachio']
dataset_names = ['Grasp_dataset_orange', 'Grasp_dataset_pear', 'Grasp_dataset_banana', 'Grasp_dataset_plum', 'Grasp_dataset_egg', 'Grasp_dataset_strawberry', 'Grasp_dataset_chicken', 'Grasp_dataset_bayberry', 'Grasp_dataset_redgrape', 'Grasp_dataset_greengrape']

header = ['pressure_reading_1', 'pressure_reading_2', 'force_reading_1', 'force_reading_2', 'p1_x', 'p1_y', 'p2_x', 'p2_y', 'p3_x', 'p3_y', 'p4_x', 'p4_y', 'p5_x', 'p5_y', 'p6_x', 'p6_y', 'p7_x', 'p7_y', 'p8_x', 'p8_y', 'p9_x', 'p9_y', 'p10_x', 'p10_y', 'p11_x', 'p11_y', 'p12_x', 'p12_y', 'p13_x', 'p13_y', 'p14_x', 'p14_y', 'p15_x', 'p15_y', 'p16_x', 'p16_y', 'label']

current_dir = os. getcwd()

pressure_reading_1_arr = []
pressure_reading_2_arr = []
force_reading_1_arr = []
force_reading_2_arr = []

p1_x_arr = []
p1_y_arr = []
p2_x_arr = []
p2_y_arr = []
p3_x_arr = []
p3_y_arr = []
p4_x_arr = []
p4_y_arr = []
p5_x_arr = []
p5_y_arr = []
p6_x_arr = []
p6_y_arr = []
p7_x_arr = []
p7_y_arr = []
p8_x_arr = []
p8_y_arr = []
p9_x_arr = []
p9_y_arr = []
p10_x_arr = []
p10_y_arr = []
p11_x_arr = []
p11_y_arr = []
p12_x_arr = []
p12_y_arr = []
p13_x_arr = []
p13_y_arr = []
p14_x_arr = []
p14_y_arr = []
p15_x_arr = []
p15_y_arr = []
p16_x_arr = []
p16_y_arr = []
label_arr = []


for f in dataset_names:
    
    image_dir = os.path.join(current_dir + '/Grasp_dataset_2', f)
    data_dir = os.path.join(image_dir, f + '.csv')
    df = pd.read_csv(data_dir)
    pressure_reading_1_arr = np.concatenate((pressure_reading_1_arr, df['pressure_reading_1'].values))
    pressure_reading_2_arr = np.concatenate((pressure_reading_2_arr, df['pressure_reading_2'].values))
    force_reading_1_arr = np.concatenate((force_reading_1_arr, df['force_reading_1'].values))
    force_reading_2_arr = np.concatenate((force_reading_2_arr, df['force_reading_2'].values))
    label_arr = np.concatenate((label_arr, df['label'].values))
    
    j = 0
    
    while j < df.shape[0]:
        image = cv2.imread(os.path.join(image_dir, df.iloc[j]['image_name']))
        keypoints = predict_keypoints(image)
        
#         plot_keypoints(image, keypoints)
        
#         if j == 20:
#             plot_keypoints(image, keypoints)
        

#         plt.imshow(image)

#         plt.scatter(kirigami_x_points*image.shape[1], kirigami_y_points*image.shape[0], s = 4, c=(1,0,0))
#         plt.show()
        
        p1_x_arr.append(keypoints[0])
        p1_y_arr.append(keypoints[1])
        p2_x_arr.append(keypoints[2])
        p2_y_arr.append(keypoints[3])
        p3_x_arr.append(keypoints[4])
        p3_y_arr.append(keypoints[5])
        p4_x_arr.append(keypoints[6])
        p4_y_arr.append(keypoints[7])
        p5_x_arr.append(keypoints[8])
        p5_y_arr.append(keypoints[9])
        p6_x_arr.append(keypoints[10])
        p6_y_arr.append(keypoints[11])
        p7_x_arr.append(keypoints[12])
        p7_y_arr.append(keypoints[13])
        p8_x_arr.append(keypoints[14])
        p8_y_arr.append(keypoints[15])
        p9_x_arr.append(keypoints[16])
        p9_y_arr.append(keypoints[17])
        p10_x_arr.append(keypoints[18])
        p10_y_arr.append(keypoints[19])
        p11_x_arr.append(keypoints[20])
        p11_y_arr.append(keypoints[21])
        p12_x_arr.append(keypoints[22])
        p12_y_arr.append(keypoints[23])
        p13_x_arr.append(keypoints[24])
        p13_y_arr.append(keypoints[25])
        p14_x_arr.append(keypoints[26])
        p14_y_arr.append(keypoints[27])
        p15_x_arr.append(keypoints[28])
        p15_y_arr.append(keypoints[29])
        p16_x_arr.append(keypoints[30])
        p16_y_arr.append(keypoints[31])
        
        j = j + 1
        
#         if j == 2:
#             break
#     break

grasp_dataset = pd.concat([pd.Series(pressure_reading_1_arr), pd.Series(pressure_reading_2_arr), pd.Series(force_reading_1_arr), pd.Series(force_reading_2_arr), pd.Series(p1_x_arr), pd.Series(p1_y_arr), pd.Series(p2_x_arr), pd.Series(p2_y_arr), pd.Series(p3_x_arr), pd.Series(p3_y_arr), pd.Series(p4_x_arr), pd.Series(p4_y_arr), pd.Series(p5_x_arr), pd.Series(p5_y_arr), pd.Series(p6_x_arr), pd.Series(p6_y_arr), pd.Series(p7_x_arr), pd.Series(p7_y_arr), pd.Series(p8_x_arr), pd.Series(p8_y_arr), pd.Series(p9_x_arr), pd.Series(p9_y_arr), pd.Series(p10_x_arr), pd.Series(p10_y_arr), pd.Series(p11_x_arr), pd.Series(p11_y_arr), pd.Series(p12_x_arr), pd.Series(p12_y_arr), pd.Series(p13_x_arr), pd.Series(p13_y_arr), pd.Series(p14_x_arr), pd.Series(p14_y_arr), pd.Series(p15_x_arr), pd.Series(p15_y_arr), pd.Series(p16_x_arr), pd.Series(p16_y_arr), pd.Series(label_arr)], axis=1, keys=header)


In [7]:
pd.options.display.max_columns = None
print(grasp_dataset.head(2))

   pressure_reading_1  pressure_reading_2  force_reading_1  force_reading_2  \
0                92.5                91.7            414.0            408.5   
1                89.7                93.3            448.0            426.0   

       p1_x      p1_y      p2_x      p2_y      p3_x      p3_y      p4_x  \
0  0.129493  0.299392  0.487687  0.019899  0.832821  0.275933  0.844225   
1  0.122529  0.312111  0.485181  0.008519  0.840467  0.268897  0.851097   

       p4_y      p5_x      p5_y      p6_x      p6_y      p7_x      p7_y  \
0  0.685107  0.495691  0.961590  0.131402  0.731573  0.444624  0.219978   
1  0.671745  0.496467  0.946734  0.129059  0.709954  0.445650  0.204257   

       p8_x      p8_y      p9_x      p9_y     p10_x     p10_y     p11_x  \
0  0.419312  0.385526  0.409821  0.521800  0.423446  0.653147  0.453503   
1  0.422302  0.371464  0.413999  0.507776  0.427952  0.640067  0.456656   

      p11_y     p12_x     p12_y     p13_x     p13_y     p14_x    p14_y  \
0  0.80093

In [8]:
grasp_dataset

,pressure_reading_1,pressure_reading_2,force_reading_1,force_reading_2,p1_x,p1_y,p2_x,p2_y,p3_x,p3_y,p4_x,p4_y,p5_x,p5_y,p6_x,p6_y,p7_x,p7_y,p8_x,p8_y,p9_x,p9_y,p10_x,p10_y,p11_x,p11_y,p12_x,p12_y,p13_x,p13_y,p14_x,p14_y,p15_x,p15_y,p16_x,p16_y,label
0,92.5,91.7,414.0,408.5,0.129493,0.299392,0.487687,0.019899,0.832821,0.275933,0.844225,0.685107,0.495691,0.961590,0.131402,0.731573,0.444624,0.219978,0.419312,0.385526,0.409821,0.521800,0.423446,0.653147,0.453503,0.800936,0.533485,0.806853,0.557456,0.651247,0.567224,0.523780,0.553662,0.391091,0.528919,0.230518,orange
1,89.7,93.3,448.0,426.0,0.122529,0.312111,0.485181,0.008519,0.840467,0.268897,0.851097,0.671745,0.496467,0.946734,0.129059,0.709954,0.445650,0.204257,0.422302,0.371464,0.413999,0.507776,0.427952,0.640067,0.456656,0.788747,0.533162,0.794575,0.556308,0.638186,0.565686,0.509950,0.552235,0.376528,0.527129,0.215127,orange
2,86.4,95.7,451.8,426.9,0.122475,0.347823,0.481754,0.027509,0.830254,0.252916,0.847161,0.705454,0.491838,0.970982,0.131146,0.714705,0.446162,0.220400,0.423450,0.386511,0.414269,0.522980,0.427328,0.655851,0.454757,0.805762,0.533072,0.808655,0.560135,0.651028,0.570745,0.523073,0.556803,0.390739,0.528044,0.231822,orange
3,79.0,102.2,463.9,425.3,0.107557,0.302716,0.468235,0.002476,0.826653,0.254414,0.841336,0.672136,0.487430,0.943943,0.116571,0.704509,0.426243,0.195612,0.402385,0.366464,0.395575,0.503291,0.411931,0.635498,0.443502,0.784461,0.524791,0.790039,0.547390,0.633745,0.555405,0.505038,0.540631,0.369965,0.514072,0.205889,orange
4,69.0,104.2,465.3,414.7,0.131039,0.290077,0.488886,0.017553,0.835291,0.279590,0.844509,0.682493,0.495833,0.958784,0.131905,0.733665,0.444823,0.218607,0.419121,0.384141,0.409516,0.520307,0.423108,0.651529,0.453244,0.799176,0.532791,0.805744,0.556277,0.650450,0.566006,0.522856,0.552760,0.389979,0.528982,0.229057,orange
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,45.3,12.6,208.1,214.8,0.120190,0.336160,0.476121,0.068123,0.835347,0.302561,0.829601,0.676747,0.483028,0.871063,0.131571,0.700474,0.443228,0.206037,0.428510,0.363306,0.422446,0.500392,0.432330,0.631766,0.451180,0.768108,0.515246,0.779895,0.530030,0.636854,0.539578,0.507240,0.530395,0.371975,0.510945,0.218741,greengrape
996,30.5,5.5,202.9,190.1,0.119524,0.328814,0.474529,0.067037,0.834599,0.303737,0.830926,0.682467,0.486476,0.855090,0.136538,0.693749,0.441406,0.202545,0.428257,0.358943,0.423699,0.496182,0.434489,0.627708,0.453457,0.762226,0.518725,0.774559,0.531759,0.634036,0.540387,0.504523,0.530623,0.368780,0.510687,0.215715,greengrape
997,12.5,23.1,188.6,188.9,0.119992,0.351200,0.472773,0.122751,0.833174,0.319224,0.838194,0.700580,0.485666,0.898540,0.130422,0.703178,0.442125,0.231621,0.432051,0.380966,0.427084,0.517064,0.436611,0.650266,0.454227,0.788937,0.519352,0.803216,0.531965,0.658300,0.538713,0.527392,0.528139,0.393292,0.508602,0.246071,greengrape
998,13.6,61.2,235.5,231.2,0.117550,0.359149,0.472808,0.123152,0.833567,0.328865,0.839199,0.695638,0.485226,0.930206,0.126019,0.720112,0.440838,0.241904,0.427655,0.393457,0.421028,0.528632,0.430981,0.661427,0.451563,0.803437,0.518155,0.816134,0.532920,0.667379,0.540056,0.536082,0.528921,0.402437,0.508226,0.254259,greengrape


In [9]:
grasp_data = grasp_dataset.drop(columns=['label'], axis=1)
# grasp_data = grasp_dataset.drop(columns=['label', 'p1_x', 'p1_y', 'p2_x', 'p2_y', 'p3_x', 'p3_y', 'p4_x', 'p4_y', 'p5_x', 'p5_y', 'p6_x', 'p6_y', 'p7_x', 'p7_y', 'p8_x', 'p8_y', 'p9_x', 'p9_y', 'p10_x', 'p10_y', 'p11_x', 'p11_y', 'p12_x', 'p12_y', 'p13_x', 'p13_y', 'p14_x', 'p14_y', 'p15_x', 'p15_y', 'p16_x', 'p16_y'], axis=1)
grasp_data['pressure_reading_1'] = grasp_data['pressure_reading_1'] / 100
grasp_data['pressure_reading_2'] = grasp_data['pressure_reading_2'] / 100
grasp_data['force_reading_1'] = grasp_data['force_reading_1'] / 1000
grasp_data['force_reading_2'] = grasp_data['force_reading_2'] / 1000
grasp_label = grasp_dataset['label']

print(grasp_data.head(2))
print(grasp_label.head(2))

   pressure_reading_1  pressure_reading_2  force_reading_1  force_reading_2  \
0               0.925               0.917            0.414           0.4085   
1               0.897               0.933            0.448           0.4260   

       p1_x      p1_y      p2_x      p2_y      p3_x      p3_y      p4_x  \
0  0.129493  0.299392  0.487687  0.019899  0.832821  0.275933  0.844225   
1  0.122529  0.312111  0.485181  0.008519  0.840467  0.268897  0.851097   

       p4_y      p5_x      p5_y      p6_x      p6_y      p7_x      p7_y  \
0  0.685107  0.495691  0.961590  0.131402  0.731573  0.444624  0.219978   
1  0.671745  0.496467  0.946734  0.129059  0.709954  0.445650  0.204257   

       p8_x      p8_y      p9_x      p9_y     p10_x     p10_y     p11_x  \
0  0.419312  0.385526  0.409821  0.521800  0.423446  0.653147  0.453503   
1  0.422302  0.371464  0.413999  0.507776  0.427952  0.640067  0.456656   

      p11_y     p12_x     p12_y     p13_x     p13_y     p14_x    p14_y  \
0  0.80093

In [25]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
from sklearn import svm
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn import tree
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier

data_train, data_test, label_train, label_test = train_test_split(grasp_data, grasp_label, test_size=0.2, random_state=np.random.randint(100))

# print(X_train.head(3))

# clf = KNeighborsClassifier(n_neighbors=6)

# knn.fit(X_train, y_train)

# y_pred = knn.predict(X_test)

# accuracy = knn.score(X_test, y_test)
# print('Accuracy:', accuracy)
# clf = RandomForestClassifier(n_estimators=1000, max_depth=50, random_state=np.random.randint(100))
clf = RandomForestClassifier(n_estimators=1000, max_depth=200, random_state=np.random.randint(100), n_jobs = -1)
# clf = HistGradientBoostingClassifier(max_iter=200, random_state=np.random.randint(100))
# clf = GradientBoostingClassifier(n_estimators=200, learning_rate=0.1, max_depth=1, random_state=0)
# clf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3)],voting='hard')

# clf = RandomForestClassifier(n_estimators=500, max_depth=50, random_state=np.random.randint(100))

num_folds = 10
cv_method = KFold(n_splits=num_folds, shuffle=True, random_state=np.random.randint(100))
cv_results = cross_val_score(clf, grasp_data, grasp_label, cv=cv_method, scoring='accuracy')

print('Cross-validation results:', cv_results)
print('Average accuracy:', cv_results.mean())

# clf.fit(data_train, label_train)
# label_pred = clf.predict(data_test)
# cm = confusion_matrix(label_test, label_pred)

# # short_dataset_names = ['orange', 'pear', 'banana', 'plum', 'egg', 'straw', 'chicken', 'bayberry', 'grape', 'pistachio']
# short_dataset_names = ['orange', 'pear', 'banana', 'plum', 'egg', 'straw', 'chicken', 'bayberry', 'rgrape', 'ggrape']


# cm_df = pd.DataFrame(cm, index=short_dataset_names, columns=short_dataset_names)

# print(cm_df)
# print(cm)

# accuracy = clf.score(data_test, label_test)
# print("Accuracy:", accuracy)

Cross-validation results: [0.91 0.9  0.87 0.88 0.9  0.84 0.9  0.88 0.93 0.94]
Average accuracy: 0.8949999999999999
